In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

def deblur_net(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    outputs = layers.Conv2D(3, 3, padding='same', activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [2]:
import tensorflow as tf 
import random
from cv2 import GaussianBlur
import numpy as np
from tensorflow.keras import datasets, layers, models

def getDataset():
  (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
  normalize = lambda x: x/x.max()
  x_train = normalize(x_train)
  x_test = normalize(x_test)
  createBlurred = lambda arr: np.array([GaussianBlur(x, (5,5), ((random.random()*3)+1)) + (0.02*np.random.normal(0,1, (32, 32, 3))) for x in arr])
  return (createBlurred(x_train), x_train), (createBlurred(x_test), x_test)

In [3]:
(X_train, y_train), (X_test, y_test) = getDataset()

170498071/170498071 [==============================] - 13s 0us/step


In [4]:
deblur_net = deblur_net((32,32,3))
deblur_net.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
deblur_net.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
1563/1563 [==============================] - 42s 18ms/step - loss: 0.0084 - accuracy: 0.6416 - val_loss: 0.0043 - val_accuracy: 0.7822
Epoch 2/100
1563/1563 [==============================] - 26s 17ms/step - loss: 0.0040 - accuracy: 0.8027 - val_loss: 0.0038 - val_accuracy: 0.8156
Epoch 3/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0036 - accuracy: 0.8122 - val_loss: 0.0033 - val_accuracy: 0.8269
Epoch 4/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0033 - accuracy: 0.8187 - val_loss: 0.0034 - val_accuracy: 0.8111
Epoch 5/100
1563/1563 [==============================] - 26s 16ms/step - loss: 0.0032 - accuracy: 0.8227 - val_loss: 0.0033 - val_accuracy: 0.8422
Epoch 6/100
1563/1563 [==============================] - 26s 17ms/step - loss: 0.0030 - accuracy: 0.8277 - val_loss: 0.0030 - val_accuracy: 0.8377
Epoch 7/100
1563/1563 [==============================] - 26s 16ms/step - loss: 0.0029 - accuracy: 0.8282 - val_loss: 0

In [12]:
# deblur_net.save_weights('./deblurCNN_weights.h5')
# !cp deblurCNN_weights.h5 /content/drive/MyDrive/